<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLDM-2022/blob/master/03-linear-classification/Classification_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Task 1 (2 points + 1 bonus point + 1 super-bonus point)

(Titanic data again, 1.0 edition before ddl)

Build a model with `sklearn`'s `LogisticRegression` to get the accuracy of at least 0.80 (0.82 for the bonus point, 0.85 for the super-bonus point) on the test set.

Some (optional) suggestions:
- Add new features (e.g. missing value indicator columns)
- Fill missing values
- Encode categorical features (e.g. one-hot encoding)
- Scale the features (e.g. with standard or robust scaler)
- Think of other ways of preprocessing the features (e.g. `Fare` $\to$ `log(Fare)`)
- Try adding polynomial features



In [1]:
import wget
url = "https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2022/main/01-intro/train.csv"
wget.download(url)

'train.csv'

In [1]:
import pandas as pd
data = pd.read_csv("train.csv", index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### About the data
Here's some of the columns
* Name - a string with person's full name
* Survived - 1 if a person survived the shipwreck, 0 otherwise.
* Pclass - passenger class. Pclass == 3 is cheap'n'cheerful, Pclass == 1 is for moneybags.
* Sex - a person's gender
* Age - age in years, if available
* SibSp - number of siblings on a ship
* Parch - number of parents on a ship
* Fare - ticket cost
* Embarked - port where the passenger embarked
 * C = Cherbourg; Q = Queenstown; S = Southampton

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

def feature_selection_and_preprocessing(dataset):
    features = dataset.copy().drop("Ticket", axis=1)
    Name_dict = {"Mr": "Mr","Miss": "Miss", "Mrs": "Mrs", "Master": "Master", "Dr": "Job", "Rev": "Royalty", "Col": "Job", "Major": "Job", "Mlle": "Miss", "Sir": "Royalty", "Lady": "Royalty", "Ms": "Mrs", "Jonkheer": "Royalty", "Capt": "Job", "Mme": "Mrs", "the Countess": "Royalty", "Don": "Royalty"}
    features["Name"] = features["Name"].apply(lambda x: x.split(',')[1].split('.')[0].strip())
    features["Name"] = features["Name"].map(Name_dict)
    features["Sex"].replace({"female": 0,"male": 1}, inplace=True)
    features["Age"] = features["Age"].fillna(features["Age"].median())
    features["Embarked"] = features["Embarked"].fillna(features["Embarked"].mode()[0])
    features["Cabin"] = features["Cabin"].fillna("U")
    features["Cabin"] = features["Cabin"].apply(lambda x: x[0])
    features["Relatives"] = features["Parch"] + features["SibSp"]
    features["Alone"] = features["Relatives"].apply(lambda x: 1 if x == 0 else 0)
    features["Generation"] = features["Relatives"].apply(lambda x: 1 if 1 <= x <= 3  else 0)
    features["Generations"] = features["Relatives"].apply(lambda x: 1 if x > 3  else 0)
    
    features[["Fare"]] = features[["Fare"]].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
    
    features = features.join(pd.get_dummies(features["Embarked"], prefix = "Embarked")).drop("Embarked", axis=1)
    features = features.join(pd.get_dummies(features["Pclass"], prefix = "Pclass")).drop("Pclass", axis=1)
    features = features.join(pd.get_dummies(features["Name"], prefix = "Name")).drop("Name", axis=1)
    features = features.join(pd.get_dummies(features["Cabin"], prefix = "Cabin")).drop("Cabin", axis=1)
    
    

    return features[["Name_Miss", "Name_Master", "Name_Job", "Name_Mr", "Pclass_2", "Relatives", "Alone", "Cabin_B", "Cabin_C", "Cabin_D", "Cabin_E", "Cabin_U", "Embarked_C", "Embarked_Q",  "Sex"]]

model = make_pipeline(
    # <YOUR CODE>
    LogisticRegression()
)


# Validation code (do not touch)
data = pd.read_csv("train.csv", index_col='PassengerId')
data_train, data_test = train_test_split(data, test_size=200, random_state=42)

model.fit(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    ),
    data_train['Survived']
)

train_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    )
)

test_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_test.drop('Survived', axis=1)
    )
)

print("Train accuracy:", accuracy_score(
    data_train['Survived'],
    train_predictions
))
print("Test accuracy:", accuracy_score(
    data_test['Survived'],
    test_predictions
))

Train accuracy: 0.8451519536903039
Test accuracy: 0.82
